In [ ]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"


In [1]:
import ray
ray.init(num_gpus=1)

2023-07-25 11:06:06,125	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.17
Ray version:,2.6.1
Dashboard:,http://127.0.0.1:8265


In [2]:
resources = ray.cluster_resources()
print(resources)

{'object_store_memory': 1715364249.0, 'node:__internal_head__': 1.0, 'node:172.19.132.220': 1.0, 'GPU': 1.0, 'memory': 3430728500.0, 'CPU': 20.0}


In [ ]:
#env = gym.make('FrankaPickPlaceFixed-v0')

In [ ]:
import gymnasium as gym
env = gym.make('FetchPickAndPlace-v2', max_episode_steps=100)

In [ ]:
observation, info = env.reset()

In [ ]:
observation

In [ ]:
info

In [ ]:
obs = observation[0]
print(obs)
obs = obs.get("observation")
print()
print(obs)

In [ ]:
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())

In [ ]:
obs

In [ ]:
type(obs.get("observation"))

In [ ]:
obsr = obs.get("observation")
obsag = obs.get("achieved_goal")
obsdg = obs.get("desired_goal")

observationFull = np.concatenate((obsr, obsag, obsdg), axis=0)
print("Full Observation:", observationFull)

In [ ]:
import numpy as np

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5])
q = np.concatenate((a, b), axis=0)
print("q = ", q)

In [ ]:
import cloudpickle
from gymnasium import Wrapper, spaces
import numpy as np
#from ray.tune.registry import register_env

# Assuming `FrankaPickPlaceRandom_v2d-v0` comes from `your_custom_environment_library`
#import gymnasium as gym

class CustomWrapper(Wrapper):
    def __init__(self, env):
        super().__init__(env)

        # Override observation space
        self.observation_space = spaces.Box(low=-10_000, high=10_000, shape=(31,), dtype=float64)
        #self.env.observation_space = spaces.Box(low=0, high=255, shape=(224, 224, 3), dtype=np.uint8)
        #self.reward_space = spaces.Box(low=-100_000, high=100_000, shape=(), dtype=np.float32)
        #self.env.rewards



    def step(self, action):
        #observation, reward, done, info = self.env.step(action)
        #observation = self.env.get_obs(update_exteroception=True)
        #observation = self.env.visual_dict.get('rgb:top_cam:224x224:2d')
        
        # Do some processing here
        obs, reward, terminated, truncated, info = self.env.step(action)
        obs_proprioception = obs.get("observation")
        obs_achieved_goal = obs.get("achieved_goal")
        obs_desired_goal = obs.get("desired_goal")

        observation = np.concatenate((obs_proprioception, obs_achieved_goal, obs_desired_goal), axis=0)

        return observation, reward, terminated, truncated, info

    def reset(self, **kwargs):
        observation, info = self.env.reset(**kwargs)

        #self.env.get_obs(update_exteroception=True)
        observation = observation.get("observation")#observation = self.env.visual_dict.get('rgb:top_cam:224x224:2d')

        return observation, info



In [ ]:
import gymnasium as gym
env = gym.make('FetchPickAndPlace-v2', max_episode_steps=100)
env = CustomWrapper(env)

In [ ]:
obs, info = env.reset()

In [ ]:
info

In [ ]:
obs

In [ ]:
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())

In [ ]:
obs

In [3]:
def env_creator(env_config={}):
    import gymnasium as gym
    env = gym.make('FetchPickAndPlace-v2', max_episode_steps=100)
    #env = CustomWrapper(env)
    env.reset()
    return env

In [4]:
from ray.tune.registry import register_env
register_env("Gymnasium_Pick_Place_0", env_creator)

In [ ]:
# Serialize with cloudpickle and save to a file
#with open("Gymnasium_Pick_Place_0.pkl", "wb") as f:
   # cloudpickle.dump(env_creator, f)

In [5]:
from ray import tune

In [6]:
tune.run("PPO",
             # algorithm specific configuration
             config={"env": "Gymnasium_Pick_Place_0",  #
                     "framework": "torch",
                     #"seed":958,
                     "num_gpus": 1,
                     "num_rollout_workers": 0,
                     #"num_envs_per_worker": 1,
                     #"num_gpus_per_worker": 0,
                     "train_batch_size": 512,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 32,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.00015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2
                     },
             local_dir="/home/cocp5/Gymnasium-Robotics-RL/Gymnasium_Pick_Place_0",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"training_iteration": 1000}
             )

2023-07-25 11:06:45,593	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/cocp5/anaconda3/envs/gmnsmRoboRL/lib/python3.9/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-07-25 11:06:48,901	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-07-25 11:06:49,018	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-07-25 11:06:49,022	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023

2023-07-25 11:06:49,231	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-25 11:06:49,232	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=547657) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=547657) 2023-07-25 11:06:52,769	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,evaluation,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_Gymnasium_Pick_Place_0_f8493_00000,512000,"{'ObsPreprocessorConnector_ms': 0.01938319206237793, 'StateBufferConnector_ms': 0.0038127899169921875, 'ViewRequirementAgentConnector_ms': 0.20714449882507324}","{'num_env_steps_sampled': 512000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 512000, 'num_agent_steps_trained': 0}",{},100,{},0,-98,-100,6,"{'sampler_results': {'episode_reward_max': -100.0, 'episode_reward_min': -100.0, 'episode_reward_mean': -100.0, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-100.0, -100.0], 'episode_lengths': [100, 100]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.4061908570671838, 'mean_inference_ms': 1.0774988539722523, 'mean_action_processing_ms': 0.14886736703515885, 'mean_env_wait_ms': 1.832672647492628, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.021028518676757812, 'StateBufferConnector_ms': 0.0044345855712890625, 'ViewRequirementAgentConnector_ms': 0.23725032806396484}}, 'episode_reward_max': -100.0, 'episode_reward_min': -100.0, 'episode_reward_mean': -100.0, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-100.0, -100.0], 'episode_lengths': [100, 100]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.4061908570671838, 'mean_inference_ms': 1.0774988539722523, 'mean_action_processing_ms': 0.14886736703515885, 'mean_env_wait_ms': 1.832672647492628, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.021028518676757812, 'StateBufferConnector_ms': 0.0044345855712890625, 'ViewRequirementAgentConnector_ms': 0.23725032806396484}, 'num_agent_steps_sampled_this_iter': 200, 'num_env_steps_sampled_this_iter': 200, 'timesteps_this_iter': 200, 'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}","{'learner': {'__all__': {'num_agent_steps_trained': 32.0, 'num_env_steps_trained': 512.0, 'total_loss': -0.010887722746701911}, 'default_policy': {'total_loss': -0.010887722746701911, 'policy_loss': -0.0076672502036672086, 'vf_loss': 6.7633465903327306e-06, 'vf_loss_unclipped': 6.7633465903327306e-06, 'vf_explained_var': 0.10745088569819927, 'entropy': 21.562073305249214, 'mean_kl_loss': 0.0060651905223494396, 'gradients_default_optimizer_global_norm': 3.683169424533844, 'curr_lr': 0.00015, 'curr_entropy_coeff': 0.00015, 'curr_kl_coeff': 0.001165799330919981}}, 'num_env_steps_sampled': 512000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 512000, 'num_agent_steps_trained': 0}",512000,0,512000,512,247.539,0,0,0,0,0,0,0,0,"{'cpu_util_percent': 6.625, 'ram_util_percent': 75.7}",{},{},{},"{'mean_raw_obs_processing_ms': 0.41266195602317746, 'mean_inference_ms': 1.0813793520842738, 'mean_action_processing_ms': 0.14865209651410577, 'mean_env_wait_ms': 1.8427607891846463, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 0.0, 'episode_reward_min': -100.0, 'episode_reward_mean': -98.0, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 6, 'policy_reward_min': {}, '

2023-07-25 11:58:04,505	INFO tune.py:1148 -- Total run time: 3078.91 seconds (3074.75 seconds for the tuning loop).


In [ ]:
sac_config = {
    "env": "RoboHive_Pick_Place_0",  # Specify your environment class here
    "framework": "torch",
    "monitor": True,
    # Add more SAC-specific config here
}


In [ ]:
import mujoco_py


In [ ]:
from ray import tune

analysis = tune.run(
    "SAC",
    config=sac_config,
    stop={"training_iteration": 100},  # Specify stopping criteria
    checkpoint_at_end=True
)


In [ ]:
import sys
print(sys.executable)


In [ ]:
!pip show mujoco-py

In [ ]:
import mujoco_py
mujoco_py.__version__

In [ ]:
!python -m ipykernel install --user --name=

In [ ]:
import ray
from ray.rllib.agents import sac

ray.init(ignore_reinit_error=True)

trainer = sac.SACTrainer(config=sac_config)
for i in range(100):
    result = trainer.train()
    print(result)


In [ ]:
env = YourRoboHiveEnv()
state = env.reset()
done = False
while not done:
    action = trainer.compute_action(state)
    state, reward, done, _ = env.step(action)
    env.render()


In [ ]:
gym.__version__

In [ ]:
import gym
from ray.tune.registry import register_env
from ray.rllib.agents.ppo import PPOTrainer
import ray

# Define your custom environment class (if not already defined)
class CustomEnv(gym.Env):
    # ...

# Create a function to initialize your environment
def custom_env_creator(env_config):
    return CustomEnv(env_config)

# Register your custom environment with RLlib
register_env("FrankaPickPlaceFixed-v0", custom_env_creator)

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Set up the configuration for your RLlib trainer
config = {
    "env": "FrankaPickPlaceFixed-v0",
    # Other configuration parameters...
}

# Train using Tune
analysis = ray.tune.run(
    "PPO",
    config=config,
    stop={"training_iteration": 10},
    checkpoint_at_end=True
)
